In [1]:
import spacy
import re
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
import textacy
import warnings
warnings.filterwarnings("ignore")

In [2]:
nlp = spacy.load("pl_core_news_sm")

In [3]:
#Czytanie stringa
text = 'To jest tutorial o przetwarzaniu języka naturalnego.'
doc = nlp(text)
print ([token.text for token in doc])


['To', 'jest', 'tutorial', 'o', 'przetwarzaniu', 'języka', 'naturalnego', '.']


In [4]:
#Czytanie pliku
with open('file.txt','r') as f:
	data = f.read()
doc = nlp(data)
print ([token.text for token in doc])

['Laccophilus', 'pictipennis', 'â€', '“', 'gatunek', 'wodnego', 'chrzÄ', '…', 'szcza', 'z', 'rodziny', 'pÅ', '‚', 'ywakowatych', 'i', 'podrodziny', 'Laccophilinae', '.']


In [5]:
#Wykrywanie zdań
text = 'Pojedynczy wirnik zapewnia tylko proste szyfrowanie szyfrem podstawieniowym. Przykładowo pin odpowiadający literze E może być połączony ze stykiem od litery T po drugiej stronie. Złożoność systemu szyfrowania polega na zastosowaniu ... wielu równoległych współosiowych wirników.'
doc = nlp(text)
for sent in doc.sents:
	print(sent, end='\n\n')

Pojedynczy wirnik zapewnia tylko proste szyfrowanie szyfrem podstawieniowym.

Przykładowo pin odpowiadający literze E może być połączony ze stykiem od litery T po drugiej stronie.

Złożoność systemu szyfrowania polega na zastosowaniu ... wielu równoległych współosiowych wirników.



In [6]:
#Tworzenie własnych separatorów
def set_custom_boundaries(doc):
	for token in doc[:-1]:
		if token.text in ['...']:
			doc[token.i+1].is_sent_start = True
	return doc

custom_nlp = spacy.load("pl_core_news_sm")
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(text)
for sent in custom_ellipsis_doc.sents:
	print(sent,end='\n\n')

Pojedynczy wirnik zapewnia tylko proste szyfrowanie szyfrem podstawieniowym.

Przykładowo pin odpowiadający literze E może być połączony ze stykiem od litery T po drugiej stronie.

Złożoność systemu szyfrowania polega na zastosowaniu ...

wielu równoległych współosiowych wirników.



In [7]:
#Tokenizacja
for token in doc:
    print (token, token.idx)

for token in doc:
    print((token, token.idx, token.text_with_ws,token.is_alpha, token.is_punct, token.is_space, token.shape_, token.is_stop),sep=', ')


Pojedynczy 0
wirnik 11
zapewnia 18
tylko 27
proste 33
szyfrowanie 40
szyfrem 52
podstawieniowym 60
. 75
Przykładowo 77
pin 89
odpowiadający 93
literze 107
E 115
może 117
być 122
połączony 126
ze 136
stykiem 139
od 147
litery 150
T 157
po 159
drugiej 162
stronie 170
. 177
Złożoność 179
systemu 189
szyfrowania 197
polega 209
na 216
zastosowaniu 219
... 232
wielu 236
równoległych 242
współosiowych 255
wirników 269
. 277
(Pojedynczy, 0, 'Pojedynczy ', True, False, False, 'Xxxxx', False)
(wirnik, 11, 'wirnik ', True, False, False, 'xxxx', False)
(zapewnia, 18, 'zapewnia ', True, False, False, 'xxxx', False)
(tylko, 27, 'tylko ', True, False, False, 'xxxx', True)
(proste, 33, 'proste ', True, False, False, 'xxxx', False)
(szyfrowanie, 40, 'szyfrowanie ', True, False, False, 'xxxx', False)
(szyfrem, 52, 'szyfrem ', True, False, False, 'xxxx', False)
(podstawieniowym, 60, 'podstawieniowym', True, False, False, 'xxxx', False)
(., 75, '. ', False, True, False, '.', False)
(Przykładowo, 77, 'Przy

In [8]:
#Tworzenie własnego tokenu
custom_nlp= spacy.load("pl_core_news_sm")
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,suffix_search=suffix_re.search,infix_finditer=infix_re.finditer,token_match=None)
custom_token_text = 'biało-czerwony'
custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_doc = custom_nlp(custom_token_text)
for token in custom_tokenizer_doc:
	print(token.text)

biało
-
czerwony


In [9]:
#Stop lista
pl_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
for stop_word in list(pl_stopwords)[:15]:
    print(stop_word)
print("Długość stoplisty: "+ str(len(pl_stopwords)))

jako
iz
każdy
twym
twoj
czasami
mimo
jezeli
przy
wy
je
moi
musi
wasza
wiele
Długość stoplisty: 381


In [10]:
#Lematyzacja
doc = nlp(text)
for token in doc:
	print(token, token.lemma_, sep=' --> ')

Pojedynczy --> pojedynczy
wirnik --> wirnik
zapewnia --> zapewniać
tylko --> tylko
proste --> prosty
szyfrowanie --> szyfrowanie
szyfrem --> szyfr
podstawieniowym --> podstawieniowym
. --> .
Przykładowo --> przykładowo
pin --> pin
odpowiadający --> odpowiadać
literze --> litera
E --> e
może --> móc
być --> być
połączony --> połączyć
ze --> z
stykiem --> styk
od --> od
litery --> litera
T --> t
po --> po
drugiej --> drugi
stronie --> strona
. --> .
Złożoność --> złożoność
systemu --> system
szyfrowania --> szyfrowania
polega --> polegać
na --> na
zastosowaniu --> zastosowanie
... --> ...
wielu --> wiele
równoległych --> równoległy
współosiowych --> współosiowy
wirników --> wirnik
. --> .


In [11]:
text = 'Jego pierwszym zespołem był klub Budowlani Jelenia Góra, w którym występował od 1949, a trenerem Marian Koczwara. Po ukończeniu szkoły średniej w 1953, został zawodnikiem drugoligowego Ogniwa Wrocław. Z wrocławską drużyną zajął w sezonie 1953/1954 pierwsze miejsce w grupie południowej II ligi, a następnie trzecie miejsce w turnieju finałowym o awans do ekstraklasy. W sezonie 1954/1955 zajął z drużyną drugie miejsce w grupie B II ligi. Od sezonu 1955/1956 był zawodnikiem pierwszoligowej Polonii Warszawa. W pierwszym sezonie w ekstraklasie zajął z drużyną piąte miejsce i zdobył tytuł króla strzelców ligi (417 pkt.). W 1957 zdobył z Polonią wicemistrzostwo, w 1959 mistrzostwo Polski i kolejny tytuł króla strzelców ligi (599 pkt.), w 1960 kolejne wicemistrzostwo Polski i trzeci tytuł króla strzelców ligi (563 pkt.). Od sezonu 1960/1961 był zawodnikiem Legii Warszawa. Z Legią zdobył mistrzostwo Polski w 1961, 1963 i 1966, wicemistrzostwo Polski w 1968, brązowy medal mistrzostw Polski w 1962, w sezonie 1963/1964 został czwarty raz w karierze najlepszym strzelcem ligi (593 pkt.).'
complete_doc = nlp(text)
words = [token.text for token in complete_doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common(4)
print("Najczęściej użyte słowa : \n",common_words)
print("Najrzadziej użyte słowa :\n",[(word,f) for word,f in word_freq.items() if f ==1])

Najczęściej użyte słowa : 
 [('ligi', 6), ('Polski', 5), ('sezonie', 4), ('miejsce', 4)]
Najrzadziej użyte słowa :
 [('zespołem', 1), ('klub', 1), ('Budowlani', 1), ('Jelenia', 1), ('Góra', 1), ('występował', 1), ('1949', 1), ('trenerem', 1), ('Marian', 1), ('Koczwara', 1), ('ukończeniu', 1), ('szkoły', 1), ('średniej', 1), ('1953', 1), ('drugoligowego', 1), ('Ogniwa', 1), ('Wrocław', 1), ('wrocławską', 1), ('1953/1954', 1), ('pierwsze', 1), ('południowej', 1), ('następnie', 1), ('trzecie', 1), ('turnieju', 1), ('finałowym', 1), ('awans', 1), ('ekstraklasy', 1), ('1954/1955', 1), ('drugie', 1), ('B', 1), ('1955/1956', 1), ('pierwszoligowej', 1), ('Polonii', 1), ('ekstraklasie', 1), ('piąte', 1), ('417', 1), ('1957', 1), ('Polonią', 1), ('1959', 1), ('kolejny', 1), ('599', 1), ('1960', 1), ('kolejne', 1), ('trzeci', 1), ('563', 1), ('1960/1961', 1), ('Legii', 1), ('Legią', 1), ('1961', 1), ('1963', 1), ('1966', 1), ('1968', 1), ('brązowy', 1), ('medal', 1), ('mistrzostw', 1), ('1962', 1

In [12]:
#Rozpoznawanie części mowy
for token in complete_doc:
	print(token, token.tag_, token.pos_, spacy.explain(token.tag_),sep=', ')



Jego, PPRON3, PRON, None
pierwszym, ADJ, ADJ, adjective
zespołem, SUBST, NOUN, None
był, PRAET, VERB, None
klub, SUBST, NOUN, None
Budowlani, SUBST, NOUN, None
Jelenia, ADJ, ADJ, adjective
Góra, SUBST, NOUN, None
,, INTERP, PUNCT, None
w, PREP, ADP, None
którym, ADJ, ADJ, adjective
występował, PRAET, VERB, None
od, PREP, ADP, None
1949, ADJ, ADJ, adjective
,, INTERP, PUNCT, None
a, CONJ, CCONJ, conjunction
trenerem, SUBST, NOUN, None
Marian, SUBST, NOUN, None
Koczwara, SUBST, NOUN, None
., INTERP, PUNCT, None
Po, PREP, ADP, None
ukończeniu, GER, NOUN, None
szkoły, SUBST, NOUN, None
średniej, ADJ, ADJ, adjective
w, PREP, ADP, None
1953, ADJ, ADJ, adjective
,, INTERP, PUNCT, None
został, PRAET, VERB, None
zawodnikiem, SUBST, NOUN, None
drugoligowego, ADJ, ADJ, adjective
Ogniwa, SUBST, NOUN, None
Wrocław, SUBST, NOUN, None
., INTERP, PUNCT, None
Z, PREP, ADP, None
wrocławską, ADJ, ADJ, adjective
drużyną, SUBST, NOUN, None
zajął, PRAET, VERB, None
w, PREP, ADP, None
sezonie, SUBST, NOUN, N

In [13]:
#filtrowanie słów po kategorii
nouns = []
adjectives = []
for token in complete_doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)

print(nouns)
print(adjectives)


[zespołem, klub, Budowlani, Góra, trenerem, Marian, Koczwara, ukończeniu, szkoły, zawodnikiem, Ogniwa, Wrocław, drużyną, sezonie, 1953/1954, miejsce, grupie, ligi, miejsce, turnieju, awans, ekstraklasy, sezonie, 1954/1955, drużyną, miejsce, grupie, B, ligi, sezonu, 1955/1956, zawodnikiem, Polonii, Warszawa, sezonie, ekstraklasie, drużyną, miejsce, tytuł, króla, strzelców, ligi, Polonią, wicemistrzostwo, mistrzostwo, Polski, tytuł, króla, strzelców, ligi, wicemistrzostwo, Polski, tytuł, króla, strzelców, ligi, sezonu, 1960/1961, zawodnikiem, Legii, Warszawa, Legią, mistrzostwo, Polski, wicemistrzostwo, Polski, medal, mistrzostw, Polski, sezonie, 1963/1964, raz, karierze, strzelcem, ligi]
[pierwszym, Jelenia, którym, 1949, średniej, 1953, drugoligowego, wrocławską, pierwsze, południowej, II, trzecie, finałowym, drugie, II, pierwszoligowej, pierwszym, piąte, 1957, 1959, kolejny, 1960, kolejne, trzeci, 1961, 1968, brązowy, 1962, czwarty, najlepszym]


In [14]:
#Wizualizacja z displaCy
about_interest_text = ('Wariabilizm – pogląd filozoficzny, według którego rzeczywistość jest zmienna, dobrze oddaje to formuła przypisywana Heraklitowi z Efezu')
about_interest_doc = nlp(about_interest_text)
displacy.render(about_interest_doc, jupyter=True,style='dep')


In [15]:
#zamiana słów na ich wersję podstawową
def is_token_allowed(token):
    if not token or not token.string.strip() or token.is_stop or token.is_punct:
        return False
    return True

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)for token in complete_doc if is_token_allowed(token)]
print(complete_filtered_tokens)

['pierwszy', 'zespół', 'kluba', 'budowlany', 'jeleni', 'góra', 'występować', '1949', 'trener', 'marianin', 'koczwara', 'ukończenie', 'szkoła', 'średni', '1953', 'zawodnik', 'drugoligowy', 'ogniwo', 'wrocław', 'wrocławski', 'drużyna', 'zająć', 'sezon', '1953/1954', 'pierwszy', 'miejsce', 'grupa', 'południowy', 'ii', 'liga', 'następnie', 'trzeci', 'miejsce', 'turniej', 'finałowy', 'awans', 'ekstraklasa', 'sezon', '1954/1955', 'zająć', 'drużyna', 'drugi', 'miejsce', 'grupa', 'b', 'ii', 'liga', 'sezon', '1955/1956', 'zawodnik', 'pierwszoligowy', 'polonia', 'warszawa', 'pierwszy', 'sezon', 'ekstraklasa', 'zająć', 'drużyna', 'piąty', 'miejsce', 'zdobyć', 'tytuł', 'król', 'strzelec', 'liga', '417', 'pkt', '1957', 'zdobyć', 'polonia', 'wicemistrzostwo', '1959', 'mistrzostwo', 'polska', 'kolejny', 'tytuł', 'król', 'strzelec', 'liga', '599', 'pkt', '1960', 'kolejny', 'wicemistrzostwo', 'polska', 'trzeci', 'tytuł', 'król', 'strzelec', 'liga', '563', 'pkt', 'sezon', '1960/1961', 'zawodnik', 'legia

In [16]:
#Ekstrakcja imienia i nazwiska (Po polsku nie działa, Imiona i nazwiska traktuje jako zwykłe rzeczowniki)
matcher = Matcher(nlp.vocab)
def extract_full_name(nlp_doc):
	pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
	matcher.add('FULL_NAME', None, pattern)
	matches = matcher(nlp_doc)
	for match_id, start, end in matches:
		span = nlp_doc[start:end]
		return span.text

name_text="Piotr Domaradzki polski pisarz, eseista, tłumacz i dziennikarz, z wykształcenia historyk, z zamiłowania grafik"
name_text_doc=nlp(name_text)
print(extract_full_name(name_text_doc))

None


In [17]:
#Ekstrakcja numeru telefonu
numberText=("Mój numer to +48 154 562 563, tylko późno nie dzwoń")

def extract_phone_number(nlp_doc):
	pattern = [{"ORTH": "+48"}, {"SHAPE": "ddd"}, {"SHAPE": "ddd"},{"SHAPE": "ddd"}]
	matcher.add('PHONE_NUMBER', None, pattern)
	matches = matcher(nlp_doc)
	for match_id, start, end in matches:
		span = nlp_doc[start:end]
		return span.text

conference_org_doc = nlp(numberText)
extract_phone_number(conference_org_doc)


'+48 154 562 563'

In [18]:
#Wyświetlanie struktury zdania
text = 'Kuba nie lubi używać js'
doc = nlp(text)
for token in doc:
    print(token.text, token.tag_,token.head.text, token.dep_)
displacy.render(doc,jupyter=True ,style='dep')

Kuba SUBST lubi nsubj
nie QUB lubi advmod
lubi FIN lubi ROOT
używać INF lubi xcomp
js SUBST używać obj


In [19]:
#Nawigacja po drzewie
one_line_about_text = ('Kuba używa Pythona i nie pracuje dla francuskiej firmy')
one_line_about_doc = nlp(one_line_about_text)
#Dzieci słowa pracuje
print([token.text for token in one_line_about_doc[5].children])
#Najbliższy sąsiad węzła pracuje
print (one_line_about_doc[5].nbor())
#Poddrzewo pracuje
print (list(one_line_about_doc[5].subtree))

['i', 'nie', 'firmy']
dla
[i, nie, pracuje, dla, francuskiej, firmy]


In [20]:
#Tworzenie zdania z poddrzewa
def flatten_tree(tree):
    return ''.join([token.text_with_ws for token in list(tree)]).strip()
print (flatten_tree(one_line_about_doc[1].subtree))


Kuba używa Pythona i nie pracuje dla francuskiej firmy


In [21]:
#detekcja grupy nominalnej(Nie działa z żadnym tekstem)
nom_text = ('Przedsięwzięcie miało służyć przekazywaniu polskich tradycji narodowych dzieciom emigrantów w okresu powstania listopadowego, najczęściej urodzonych we Francji w rodzinach mieszanych polsko-francuskich.')
nom_doc = nlp(nom_text)
for chunk in nom_doc.noun_chunks:
	print(chunk)

In [22]:
#Detekcja grupy czasownikowej
about_talk_text=('W początkach roku 1878, kiedy świat polityczny zajmował się pokoగem san-stefańskim,wyborem nowego papieża')
pattern = [{"POS": "VERB", "OP": "*"},{"POS": "ADV", "OP": "*"},{"POS": "VERB", "OP": "+"},{"POS": "PART", "OP": "*"}]
about_talk_doc = textacy.make_spacy_doc(about_talk_text,lang='pl_core_news_sm')
verb_phrases = textacy.extract.matches(about_talk_doc, pattern)
for chunk in verb_phrases:
    print(chunk.text)

for chunk in about_talk_doc.noun_chunks:
    print (chunk)

zajmował
zajmował się


In [23]:
#Wykrywanie słów mających jakieś znaczenie
football_class_text=('Klub piłkarski Stomil znajduje się w Olsztynie i w następnej kolejce grają z drużyną z Lublina')
football_class_doc = nlp(football_class_text)
for ent in football_class_doc.ents:
    print(ent.text, ent.start_char, ent.end_char,ent.label_, spacy.explain(ent.label_), sep=", ")

displacy.render(football_class_doc,jupyter=True ,style='dep')

Stomil, 15, 21, persName, None
Olsztynie, 37, 46, placeName, None
Lublina, 87, 94, placeName, None


In [24]:
#Używanie NER do wyszukiwania imion i nazwisk
survey_text =('Adam Kowalski i Edyta Marcinkiewicz lubią jabłka, a Kamil Nowak woli pomarańcze')
def replace_person_names(token):
	if token.ent_iob != 0 and token.ent_type_ == 'persName':
		return '[CENZURA] '
	return token.string

def redact_names(nlp_doc):
	for ent in nlp_doc.ents:
		ent.merge()
	tokens = map(replace_person_names, nlp_doc)
	return ''.join(tokens)

survey_doc = nlp(survey_text)
redact_names(survey_doc)


'[CENZURA] i [CENZURA] lubią jabłka, a [CENZURA] woli pomarańcze'